# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a classification example. This example is derived from [Hyperparameter Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use Tune to execute your training hyperparameter sweep, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#summary) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine or analyse the results returned
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +1.4s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.4s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.5s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +4.0s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-19 10:41:36,250	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-19 10:41:36,289	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_84b2e1e83156b334.zip' (6.24MiB) to Ray cluster...
2022-01-19 10:41:42,802	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_84b2e1e83156b334.zip'.


(anyscale +23.0s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +23.0s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial with a set of unique config from the search space
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=21141) == Status ==
(run pid=21141) Current time: 2022-01-19 10:43:23 (running for 00:00:00.12)
(run pid=21141) Memory usage on this node: 3.1/61.4 GiB
(run pid=21141) Using FIFO scheduling algorithm.
(run pid=21141) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/215.8 GiB heap, 0.0/91.38 GiB objects
(run pid=21141) Result logdir: /home/ray/ray_results/train_func_model_2022-01-19_10-43-22
(run pid=21141) Number of trials: 4/4 (4 PENDING)
(run pid=21141) 
(run pid=21141) 


(ImplicitFunc pid=1804, ip=172.31.94.12) 2022-01-19 10:43:26,152	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1992, ip=172.31.93.3) 2022-01-19 10:43:26,316	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1749, ip=172.31.64.127) 2022-01-19 10:43:26,287	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=21484) 2022-01-19 10:43:26,392	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1804, ip=172.31.94.12) 2022-01-19 10:43:28,168	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1838, ip=172.31.94.12) [10:43:28] task [xgboost.ray]:140670470906496 got new rank 2
(_RemoteRayXGBoostActor pid=1836, ip=172.31.94.12) 

(run pid=21141) 2022-01-19 10:43:29,233	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=21141) 2022-01-19 10:43:29,233	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
(run pid=21141) Authenticating
(run pid=21141) Loaded Anyscale authentication token from variable.
(run pid=21141) 
(run pid=21141) 2022-01-19 10:43:30,891	INFO command_runner.py:357 -- Fetched IP: 172.31.94.12
(run pid=21141) 2022-01-19 10:43:30,891	INFO log_timer.py:25 -- NodeUpdater: ins_vxY19qqMFxr4ZUuJWudfvS75: Got IP  [LogTimer=271ms]
(run pid=21141) == Status ==
(run pid=21141) Current time: 2022-01-19 10:43:31 (running for 00:00:07.95)
(run pid=21141) Memory usage on this node: 3.9/61.4 GiB
(run pid=21141) Using FIFO scheduling algorithm.
(run pid=21141) Resources requested: 20.0/80 CPUs, 0/0 GPUs, 0.0/215.8 GiB heap, 0.0/91.38 GiB objects
(run pid=21141) Current best trial: adf81_00000 with train-error=0.024605 and 

(ImplicitFunc pid=21484) 2022-01-19 10:43:36,464	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.12 seconds (7.74 pure XGBoost training time).
(ImplicitFunc pid=1804, ip=172.31.94.12) 2022-01-19 10:43:36,455	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.35 seconds (8.28 pure XGBoost training time).
(ImplicitFunc pid=1992, ip=172.31.93.3) 2022-01-19 10:43:36,468	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.20 seconds (7.93 pure XGBoost training time).
(ImplicitFunc pid=1749, ip=172.31.64.127) 2022-01-19 10:43:36,461	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.22 seconds (8.05 pure XGBoost training time).
(run pid=21141) 2022-01-19 10:43:36,592	INFO tune.py:626 -- Total run time: 13.66 seconds (12.68 seconds for the tuning loop).


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.06582567876104713, 'subsample': 0.8952836821935377, 'max_depth': 6}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/envs/ray-core/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
adf81_00000,0.690572,0.012302,0.006305,True,None,None,10,99470c665ed544978fbc44d7f0f98cda,2022-01-19_10-43-36,1642617816,...,10,"0_eta=0.00029617,max_depth=6,subsample=0.82279",approx,binary:logistic,"[logloss, error]",0.000296,0.822787,6,1,1
adf81_00001,0.334131,0.010545,0.013719,True,None,None,10,8dd10e255c1a4f539852bb3b03d7b92a,2022-01-19_10-43-36,1642617816,...,10,"1_eta=0.065826,max_depth=6,subsample=0.89528",approx,binary:logistic,"[logloss, error]",0.065826,0.895284,6,1,1
adf81_00002,0.666818,0.014060,0.006467,True,None,None,10,65fca4828a25455bac5de3d11a725f70,2022-01-19_10-43-36,1642617816,...,10,"2_eta=0.0031309,max_depth=8,subsample=0.76431",approx,binary:logistic,"[logloss, error]",0.003131,0.764307,8,1,1
adf81_00003,0.659068,0.042179,0.008621,True,None,None,10,31cb6eab18db4b39b1eaa9c1a5aea708,2022-01-19_10-43-36,1642617816,...,10,"3_eta=0.0043735,max_depth=2,subsample=0.53375",approx,binary:logistic,"[logloss, error]",0.004374,0.533747,2,1,1


---

In [9]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)